#my ssim

In [ ]:
!pip install tensorflow-addons

import keras.backend as K
import tensorflow_addons as tfa
import math

def my_ssim(y_true, y_pred):

  #source: https://www.tensorflow.org/addons/api_docs/python/tfa/image/gaussian_filter2d
  #Means obtained by Gaussian filtering of inputs => source: https://www.programcreek.com/python/?CodeExample=compute+ssim
  #Sigma is a standard deviation of the gaussian

  y_true_mean = tfa.image.gaussian_filter2d(y_true, filter_shape=(11,11), sigma=1.5) #input and output is a Tensor
  y_pred_mean = tfa.image.gaussian_filter2d(y_pred, filter_shape=(11,11), sigma=1.5)

  #https://docs.w3cub.com/tensorflow~2.3/image/psnr
  #https://github.com/keras-team/keras/issues/1917
  #convert Tensorflow tensor to numpy
  y_true_mean = K.cast( y_true_mean, 'float32')/255
  y_pred_mean = K.cast( y_pred_mean, 'float32')/255

  y_true = K.cast( y_true, 'float32')/255
  y_pred = K.cast( y_pred, 'float32')/255

  #source: https://github.com/mubeta06/python/blob/master/signal_processing/sp/ssim.py
  # k1 & c1 depend on L (width of color map)
  l = 255
  k_1 = 0.01
  k_2 = 0.03
  c_1 = (k_1 * l)**2
  c_2 = (k_2 * l)**2

  # Squares of means
  mu_1_sq = y_true_mean**2
  mu_2_sq = y_pred_mean**2
  mu_1_mu_2 = y_true_mean * y_pred_mean

  sigma1_sq = (y_true - y_true_mean)**2
  sigma1_sq_mean = (tfa.image.mean_filter2d((y_true - y_true_mean)**2))**1/2
  sigma2_sq = (y_pred - y_pred_mean)**2 
  sigma2_sq_mean = (tfa.image.mean_filter2d((y_pred - y_pred_mean)**2))**1/2
  sigma12 = tfa.image.mean_filter2d((y_true - y_true_mean) * (y_pred - y_pred_mean))

  ssim_map = ((2 * mu_1_mu_2 + c_1)*(2 * sigma12 + c_2)) / ((mu_1_sq + mu_2_sq + c_1)*(sigma1_sq + sigma2_sq + c_2))
 
  # return MSSIM
  index = K.mean(ssim_map)

  return index

#skimage ssim

In [ ]:
from skimage.metrics import structural_similarity

def sk_ssim(y_true , y_pred):
  return structural_similarity(y_true , y_pred, guassian_weights=True , multichannel=True)

#tensorflow ssim

In [ ]:
from tensorflow.image import ssim

def tf_ssim(y_true , y_pred):
  return ssim(y_true , y_pred ,  max_val=255)

#downsample ssim

In [ ]:
!pip install git+https://github.com/olivesgatech/dippykit.git

from dippykit.metrics import SSIM as dsample_ssim

def downsample_ssim(y_true , y_pred):
  return dsample_ssim(y_true, y_pred, multichannel=True, auto_downsample = True, data_range=255.0)[0]